In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)


## 사이즈사전 환경설정
##pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install py-hanspell
# !pip install PyKomoran
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from selenium import webdriver
import pandas as pd
from tqdm.notebook import tqdm
import time, urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
keys = Keys()
import numpy as np;import re, os, sys, json

from hanspell import spell_checker

from konlpy.tag import Kkma #형태소분석
from konlpy.tag import Okt #형태소분석
import soynlp; from soynlp.normalizer import * #정규화

from konlpy.tag import Komoran, Hannanum, Kkma, Okt
komoran = Komoran(); hannanum = Hannanum(); kkma = Kkma(); okt = Okt();
# from pykospacing import Spacing
# spacing = Spacing()

In [7]:
df_origin = pd.read_csv('data/hood_page1to20_final_nodup.csv')
df = df_origin.copy()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35752 entries, 0 to 35751
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   35752 non-null  int64  
 1   user         35752 non-null  object 
 2   gender       33350 non-null  object 
 3   height       33350 non-null  object 
 4   weight       33350 non-null  object 
 5   item         35752 non-null  object 
 6   size         35752 non-null  object 
 7   star         35752 non-null  float64
 8   content      35752 non-null  object 
 9   size_eval    35183 non-null  object 
 10  bright_eval  35183 non-null  object 
 11  color_eval   35183 non-null  object 
 12  thick_eval   35183 non-null  object 
 13  cm           21905 non-null  object 
 14  총장           21905 non-null  float64
 15  어깨너비         20885 non-null  float64
 16  가슴단면         21905 non-null  float64
 17  소매길이         21798 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory usa

In [18]:
print('null값모두삭제:{}'.format(df.dropna(axis = 0).shape))
print('성별,키,몸무게 null 삭제:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()].shape))
print('성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())].shape))
print('성별,키,몸무게 null:{}'.format(df[df['gender'].isnull()&df['height'].isnull()&df['weight'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이4개 전부 null:{}'.format(df[df['총장'].isnull()&
   df['어깨너비'].isnull()&df['가슴단면'].isnull()&
   df['소매길이'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이 하나라도 null:{}'.format(df[df['총장'].isnull()|
   df['어깨너비'].isnull()|df['가슴단면'].isnull()|
   df['소매길이'].isnull()].shape))



null값모두삭제:(19511, 18)
성별,키,몸무게 null 삭제:(33350, 18)
성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:(20467, 18)
성별,키,몸무게 null:(2402, 18)
총장,어깨너비,가슴단면,소매길이4개 전부 null:(13847, 18)
총장,어깨너비,가슴단면,소매길이 하나라도 null:(14967, 18)


In [20]:
df_figure = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
df_figure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20467 entries, 100 to 35737
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   20467 non-null  int64  
 1   user         20467 non-null  object 
 2   gender       20467 non-null  object 
 3   height       20467 non-null  object 
 4   weight       20467 non-null  object 
 5   item         20467 non-null  object 
 6   size         20467 non-null  object 
 7   star         20467 non-null  float64
 8   content      20467 non-null  object 
 9   size_eval    20467 non-null  object 
 10  bright_eval  20467 non-null  object 
 11  color_eval   20467 non-null  object 
 12  thick_eval   20467 non-null  object 
 13  cm           20467 non-null  object 
 14  총장           20467 non-null  float64
 15  어깨너비         19611 non-null  float64
 16  가슴단면         20467 non-null  float64
 17  소매길이         20366 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory u

In [24]:
df_figure.iloc[: , 1:].to_csv('data/hoodtrain_sample.csv',index=False,encoding = "UTF-8")

In [61]:
df1 = pd.read_csv('data/hood_2_1_11page.csv')
df2 = pd.read_csv('data/hood_2_12_30page.csv')
df3 = pd.read_csv('data/hood_2_31_40page.csv')
df4 = pd.read_csv('data/hood_2_41_70page.csv')
df4.drop(['허리단면','허벅지단면','밑위','밑단단면'],axis = 1,inplace=True)
df5 = pd.read_csv('data/hood_2_71_90page.csv')
dfpage2 = pd.concat([df1,df2,df3,df4,df5])
dfpage2.to_csv('data/hood_2page.csv',encoding="UTF-8", index=False)



## 전처리에 들어갈 코드

* 사이즈 사전완성하고 돌릴 df완성

In [108]:
def crawlingdataconcat():
    df = pd.DataFrame()
    for i in range(1,21,1):
        try:
            df_before = pd.read_csv('data/hood_'+str(i)+'page.csv') #크롤링파일 불러오기
            print('df',str(i),': ',df_before.shape)
        except:
            df_before = pd.DataFrame() #아직크롤링되지 않은파일은 빈 df
            print('df',str(i),': ',df_before.shape)
            
        df = pd.concat([df,df_before]) #합치기
            

            
    print('df : ', df.shape)
    df.drop_duplicates(subset = None,keep = 'first', inplace = True,ignore_index = True)
    print('중복제거df : ', df.shape)#전체열이 같은 중복제거
    #성별, 키,몸무게 모두 null아니고 수치종류 적어도 하나 값이 있는 것들 filter
    df = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
    print('최종사용 data:{}'.format(df.shape))
    print('-'*10)
    return df

def crawlingdataprocessing(df):
    #kg,cm제거 및 수치타입으로 변경
    df["height"]=df["height"].replace('cm','',regex = True)
    df["weight"]=df["weight"].replace('kg','',regex = True)
    df= df.astype({'height':'float','weight':'float'})
    print('df전처리완료')
    print('-'*10)
    return df

def add_reviewcol(df):
    df = df.drop_duplicates(subset=['content'])#리뷰중복제거
    print('같은내용리뷰 중복제거 완료')
    df.reset_index(drop=True, inplace=True)

    df['review'] = str('')
    #외국어 리뷰 삭제
    i = 0; nohangul = []
    for i in range(df.shape[0]):
        text = re.sub('[^ㄱ-ㅣ가-힣]', '',df.iloc[i,8])
        if(text==''):
            nohangul.append(i)
    df = df.iloc[[True if i not in nohangul else False for i in range(df.shape[0])],:]
    df.reset_index(drop=True, inplace=True)

    i=0
    for i in range(df.shape[0]):
        text = df.iloc[i,7]
        text = re.sub(pattern='[^\w\s\n]', repl='', string=text) #특수문자 제거
        text = re.sub(pattern='[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z]', repl=' ', string=text) #숫자, 그이외 삭제
        text = re.sub(pattern='[ㄱ-ㅎㅏ-ㅣ]+', repl='', string=text) #단순 모음, 자음 삭제
        text = repeat_normalize(text, num_repeats=2) #불필요반복문자정규화
        #text = spacing(text) #띄어쓰기------------------------>써야함!
        df['review'][i] = text
    print('리뷰 전처리 열 추가')
    print('df shape:{}'.format(df.shape))
    print('-'*10)
    return df

df1 = crawlingdataconcat()
df2 = crawlingdataprocessing(df1)
df3 = add_reviewcol(df2)

df 1 :  (9000, 17)
df 2 :  (8808, 17)
df 3 :  (8370, 17)
df 4 :  (6690, 17)
df 5 :  (7649, 17)
df 6 :  (6419, 17)
df 7 :  (5687, 17)
df 8 :  (5715, 17)
df 9 :  (5102, 17)
df 10 :  (5154, 17)
df 11 :  (4318, 17)
df 12 :  (3671, 17)
df 13 :  (3389, 17)
df 14 :  (3673, 17)
df 15 :  (3103, 17)
df 16 :  (2906, 17)
df 17 :  (2658, 17)
df 18 :  (2445, 17)
df 19 :  (1903, 17)
df 20 :  (1734, 17)
df :  (98394, 17)
중복제거df :  (39203, 17)
최종사용 data:(22502, 17)
----------
df전처리완료
----------
같은내용리뷰 중복제거 완료
리뷰 전처리 열 추가
df shape:(20037, 18)
----------


In [ ]:
#최종 data = df

### 사이즈사전

In [5]:
#total,chongjang,shoulder,chest,arm,small,big
total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏이','핏','핏입니다',
      '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
      '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷','후드','후드티','핏은','좋아요핏','요핏도',
      '좋아요핏도','상의','요핏','폼','폭','사이즈','후드','옷','품','전체','품도','몸통','옷핏','상체','핏',
      '핏이구','핏이었','핏임','핏입',
      '핏입니','핏감','요핏','폼','후드핏','핏이','크기','상품','상의','핏더']
chongjang = ['길이','기장','총장','궁디','궁딩','엉덩이','밑','밑단','위아래','끝단','밑기장',
             '밑단이랑','기장이','기장입니','밑단','밑','기장감','요기장','총길이','총장','총기장',
            '길이감','끝','기장','길이']
shoulder = ['어깨핏','어께','어깨','너비','어깡','골격','바디','가로','넓이','등판','통',
            '어깨깡패','어깨라','어깨넓','어깨쪽','어께','몸집','어깨핏','어깨선','어깨라인','어깡',
            '어깨','통','몸통','골격','넓이','너비','등판']

chest = ['가슴','둘레','바디','가로','넓이','통','둘레','몸집','가슴팍','통','몸통']
arm = ['소매','팔도','손목','손','팔다리','팔목','팔','팔꿈치','팔이','요팔',
       '당팔','팔길이','팔다리','팔길','팔소매','손목','팔목','소매통','손','팔','소매','팔기장']

small = ['작습니다','짧았어요','짧습니다','짧으면','짧','작았어요','짧다고','쪼여서','작았으면',
      '작아도','작으면','작음','작아진','좁다고',
      '짧아요','짧게','짧다는','좁다는','작네','작기는','달라붙는','작은데','짧은데','짧음',
      '작긴','짧네요','작게','작아서','작은','짧고','짧아','좁은','타이트','짧았지만','붙어서','짧긴',
      '작은것','작아요','짧은','짧아서','짧지만','좁아','좁고','작네요','작아','쪼이는','좁아요','크롭느낌',
      '크롭하','작고','숏','쫄려','작다고','작다','좁아서','작','크롭','크롭된','미니','크롭해',
      '길었으면','컸으면','좁게','짧지' ,'작지','크롭이','핏되',
      '숏하','사이즈업하','크롭하','숏한','크롭하긴','크롭이','크롭해','크롭느낌','크롭된','사이즈업','짝',
       '작','숏','짧긴','크롭','크롭한','타이트','크롭이라','작긴',
       '작','짧','좁']

big = ['큰','긴','커요','크고','크긴','넉넉하고','길어서','넉넉하게','작다는'
     '넉넉한','클','길고','큰데','크지','크지만','부해','크니깐','크더라구요','컸어도','넓음','큼직하게',
     '박시하네요','넉넉할','헐렁하게','큼직하니','컸음','큽니','긴데','길어','길게','넓어서','넓어','박시하고','컸어요',
     '박시','크다고','크다는','넓게','넓고','커용','길다는','크지도','큼','크다','길었어요','넉넉함','큰데도','덮는데',
     '덮이고','컸고','벙벙해서','크더라고요','컸을','크거나','크더라','컸었는데','컸어용','컸습니다','덮네요',
     '큼지막해서','덥는','박시하게','크네요','클까','넉넉하구','박시하','컸는데','넉넉하네요','크게','크니까','넉넉한데','길어요',
     '컸지만','헐렁한','크네','넉넉해','펑퍼짐한','덮어용','박시하니','헐렁해','넉넉하지''넉넉하구요','커','큽니다','넉넉하니',
     '크','길지','박시한데','덮어요','길긴','넉넉해요','넓은','덮히는', '넓긴','덮어줘서','덮은','넉넉해서','큼직해서','기네','낙낙',
     '흘러내리는','덮어서','벙벙한','커서','큰것','박시핏','오퍼핏','낙낙해','여유핏','널널함','길','덮어','넉넉합니다','크면',
     '덮고','가오리','루즈핏','펑퍼짐','길었지만','헐렁하고','남아','아방핏','박스핏','널널','오바핏','덮습니다','넉넉히',
     '헐렁해서','접어도','접어','접어서','널','넓어요','덮고도','오버','와이드','낭낭','헐렁','접으면','짧았으면','접고',
     '버핏','접어야','잡아먹힌','수선해서','오버핏','오버사이즈','오버사이징해서','낙낙하다','낙낙한데','낙낙해요','오버핏이예요','오버핏입니당',
       '요박시하','컸어용','낙낙합니','어벙벙하','와이드핏','크니깐','크네요옷','널널해요','오버핏나와요','널럴하',
       '요넉넉하','엄청큼','여유핏','오버느낌','아방핏','오버핏은','넓긴','넉넉함','입니다오버핏','큽니다오버핏',
       '큽니다제','큽니당','오버해요','요박시한','헐렁하긴','헐렁거리','아방방한','아방방','박시합니','박시했다',
       '빅사이즈','받았습니다오버핏','빡시하','루즈핏으로','낭낭한','오버핏이긴','박스핏','널널함','어벙벙',
       '요넉넉한','오퍼핏','오버핏이구요','낙낙해','낭낭해서','넉넉하구','넉넉한데','아방해','오버핏이라서',
       '낭낭하','박시해','커용','넉넉해','펑퍼짐','박시하지','널널','좋아요오버핏','어벙벙한','널널하','오바핏',
       '오버하지','오버핏하','박시합니다','오버핏되','박시함','버핏','박시한','낙낙하니','박시해요','크네용',                     '낙낙해서','큼직해서','루즈해','느슨','널널합니다','아방아','아방아방','루즈핏이','박시하','세미오버',
       '세미오버핏','루즈해서','오버핏이라','루즈핏이라','오버핏이네요','오버핏입니','어벙','오버해','박시한데',
       '아방','오버핏이에요','예뻐요오버핏','큼지막','박시핏','오버해서','루즈한','오버핏인데','넉넉하구요','박시할',
       '큼직','오버한','오버핏이','낙낙하','아방하','롱','오버핏으로','낙낙한','오버핏입니다','박시해서','와이드',                     '요오버핏','넉넉해서','큽니','오버','오버하','루즈','루즈핏','박시','넉넉','길','크긴','아방한','널널한',
       '헐렁','길긴','와이드하','아방해서','세미오버사이즈',             
       '크','길','널','덮','넓','벙벙하','덮이','덮히','커다랗','덥히','넉넉해지','뒤집'             
       '넉넉']

In [29]:
# 두개의 사전이 합치고 나서 진행
# 중복제거
# 'n+2'개의 최종 리스트가 생성됨
def sizedict_nodup(oneOfRawList):
    print('중복제거 전:{}개'.format(len(oneOfRawList)))
    oneOfRawList = set(oneOfRawList)
    print('중복제거 후:{}개'.format(len(oneOfRawList)))
    print('-'*10)
    return list(oneOfRawList)

In [30]:
total = sizedict_nodup(total)
chongjang = sizedict_nodup(chongjang)
shoulder = sizedict_nodup(shoulder)
arm  = sizedict_nodup(arm)
chest = sizedict_nodup(chest)
small = sizedict_nodup(small)
big = sizedict_nodup(big)


중복제거 전:52개
중복제거 후:52개
----------
중복제거 전:20개
중복제거 후:20개
----------
중복제거 전:19개
중복제거 후:19개
----------
중복제거 전:16개
중복제거 후:16개
----------
중복제거 전:9개
중복제거 후:9개
----------
중복제거 전:76개
중복제거 후:76개
----------
중복제거 전:255개
중복제거 후:255개
----------


### 사전으로 01추출해보기 여기부터 해보기 마구잡이

In [152]:
#total,chongjang,shoulder,chest,arm ->keywords리스트

def get_keywords(keywords, df,keyword_column_name):
    start = time.time()
    review = df['review']
    df['keyword_column_name'] = str('')
    for i in tqdm(range(len(review))):
        # if i % 1000 ==0:
        #     print("{}번째 리뷰 완료(천 단위) : ".format(i))
        keywords_search = []
        for j in keywords:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                #bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' +' '+ j ,review[i])
                keywords_search.extend(a)
                keywords_search.extend(aa)
                keywords_search.extend(b)
                keywords_search.extend(bb)
                # print(i,'번째',j,a,end='|')
                # print(aa,end='|')
                # print(b,end='|')
                # print(bb)
        #print(i,'번째 리스트:',keywords_search)
        #print('-'*10)
        
        if len(keywords_search) != 0:
            keywords_o = ','.join(x for x in keywords_search)
            df['keyword_column_name'][i] = keywords_o 
            
        else:
            df['keyword_column_name'][i] = '0'
    
    df.rename(columns = {'keyword_column_name':keyword_column_name},inplace=True)
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return df
                
             

    

In [153]:
#total,chongjang,shoulder,chest,arm ->keywords리스트
keyword_column_name = 'total_keyword'
df_total = get_keywords(total,df3.head(2000),keyword_column_name)
keyword_column_name = 'chongjang_keyword'
df_chongjang = get_keywords(chongjang,df3.head(2000),keyword_column_name)
keyword_column_name = 'shoulder_keyword'
df_shoulder = get_keywords(shoulder,df3.head(2000),keyword_column_name)
keyword_column_name = 'chest_keyword'
df_chest = get_keywords(chest,df3.head(2000),keyword_column_name)
keyword_column_name = 'arm_keyword'
df_arm = get_keywords(arm,df3.head(2000),keyword_column_name)
df_keywords = pd.concat([df_total,df_chongjang['chongjang_keyword'],
                         df_chest['chest_keyword'],df_shoulder['shoulder_keyword'],
                         df_arm['arm_keyword']],axis = 1)
df_keywords.info()
# total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏이','핏','핏입니다',
#       '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
#       '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷','후드','후드티','핏은','좋아요핏','요핏도',
#       '좋아요핏도','상의','요핏','폼','폭','사이즈','후드','옷','품','전체','품도','몸통','옷핏','상체','핏',
#       '핏이구','핏이었','핏임','핏입',
#       '핏입니','핏감','요핏','폼','후드핏','핏이','크기','상품','상의','핏더']
#0,1,5,7,9,11,12,13,15,17,18,19,

100%|██████████| 2000/2000 [00:02<00:00, 728.45it/s] 


2.75352 sec


100%|██████████| 2000/2000 [00:01<00:00, 1238.47it/s]


1.62218 sec


100%|██████████| 2000/2000 [00:02<00:00, 875.63it/s] 


2.29686 sec


100%|██████████| 2000/2000 [00:01<00:00, 1614.62it/s]


1.24575 sec


100%|██████████| 2000/2000 [00:01<00:00, 1416.21it/s]

1.42012 sec
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user               2000 non-null   object 
 1   gender             2000 non-null   object 
 2   height             2000 non-null   float64
 3   weight             2000 non-null   float64
 4   item               2000 non-null   object 
 5   size               2000 non-null   object 
 6   star               2000 non-null   float64
 7   content            2000 non-null   object 
 8   size_eval          2000 non-null   object 
 9   bright_eval        2000 non-null   object 
 10  color_eval         2000 non-null   object 
 11  thick_eval         2000 non-null   object 
 12  cm                 2000 non-null   object 
 13  총장                 2000 non-null   float64
 14  어깨너비               1804 non-null   float64
 15  가슴단면               2000 non-null   float64
 16  소매길이        

In [156]:
df_keywords[df_keywords['arm_keyword']!='0'].head()

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이,review,total_keyword,chongjang_keyword,chest_keyword,shoulder_keyword,arm_keyword
12,LV 2 뉴비_c6c71c6b525c,남성,177.0,62.0,SIGN LOGO HOOD GRAY,M,5.0,티셔츠 재질이 좋아요청바지랑 슬랙스 모두 잘 어울려요손이자주가는 옷중하나입니다유행안타고 잘입을수있을꺼 같아요,보통이에요,보통이에요,보통이에요,보통이에요,M,70.0,63.0,59.0,55.0,티셔츠 재질이 좋아요청바지랑 슬랙스 모두 잘 어울려요손이자주가는 옷중하나입니다유행안타고 잘입을수있을꺼 같아요,"옷중하나입니다유행안타고 잘입을수있을꺼 같아요,어울려요손이자주가는 옷",0,0,0,"손이자주가는 옷중하나입니다유행안타고 잘입을수있을꺼,어울려요손"
114,LV 3 뉴비_0735177a2068,남성,176.0,62.0,(23SS) 2 TONE ARCH HOODIE BLACK,L,4.0,되게 편하고 좋아요손이 잘 가고 무난한거같아요,커요,보통이에요,보통이에요,보통이에요,L,69.0,60.0,61.0,63.0,되게 편하고 좋아요손이 잘 가고 무난한거같아요,0,0,0,0,"손이 잘 가고,좋아요손"
126,LV 4 뉴비_0d59490f6282,남성,173.0,70.0,(23SS) 2 TONE ARCH HOODIE GRAY,M,5.0,l은 클거같아서 m 샀는데 반팔 입고 바로 입으면 딱 맞는 정도에요 l 정핏은 m 가고 오버핏은 l 가면 될 듯,작아요,보통이에요,보통이에요,보통이에요,M,67.0,58.0,59.0,62.0,l은 클거같아서 m 샀는데 반팔 입고 바로 입으면 딱 맞는 정도에요 l 정핏은 m 가고 오버핏은 l 가면 될 듯,"정핏은,오버핏은,정핏,오버핏",0,0,0,"팔 입고 바로 입으면,반팔"
162,LV 4 babobabo,남성,176.0,87.0,(23SS) 2 TONE ARCH HOODIE BLACK,XL,5.0,사이즈괜찬아요Xl인데 기장품 어깨 소매길이 모두 적당합니다,보통이에요,보통이에요,선명해요,보통이에요,XL,71.0,62.0,63.0,64.0,사이즈괜찬아요Xl인데 기장품 어깨 소매길이 모두 적당합니다,"품 어깨 소매길이 모두,기장품","기장품 어깨 소매길이,인데 기장,소매길이",0,"어깨 소매길이 모두 적당합니다,기장품 어깨","소매길이 모두 적당합니다,어깨 소매"
168,LV 3 너구리쭈꾸미,여성,167.0,48.0,[Mmlg] 19MG HOOD (NATURAL SOAP),M,5.0,예뻐요컬러감도 따뜻하고여성에게 잘 맞는거 같아요길이감이 좀 짧게ㅈ나오고 소매는 긴편입니다,보통이에요,밝아요,선명해요,보통이에요,M,67.0,56.0,63.0,64.0,예뻐요컬러감도 따뜻하고여성에게 잘 맞는거 같아요길이감이 좀 짧게나오고 소매는 긴편입니다,0,"길이감이 좀 짧게나오고,같아요길이,길이감이 좀 짧게나오고,같아요길이감",0,0,짧게나오고 소매


In [145]:
len(df_keywords['total_keyword'])

2000

## big, small 감지

In [163]:
#small,big
def total_big_small(big,small,df):
    start = time.time()
    review = df['total_keyword']
    df['total_big'] = str('')
    df['total_small'] = str('')
    for i in tqdm(range(len(review))):
        # if i % 1000 ==0:
        #     print("{}번째 리뷰 완료(천 단위) : ".format(i))
        big_search = []
        small_search = []
        for j in big:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                #bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' +' '+ j ,review[i])
                big_search.extend(a)
                big_search.extend(aa)
                big_search.extend(b)
                big_search.extend(bb)
                # print(i,'번째',j,a,end='|')
                # print(aa,end='|')
                # print(b,end='|')
                # print(bb)
        #print(i,'번째 리스트:',keywords_search)
        #print('-'*10)
        for j in small:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                #bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' +' '+ j ,review[i])
                small_search.extend(a)
                small_search.extend(aa)
                small_search.extend(b)
                small_search.extend(bb)
                # print(i,'번째',j,a,end='|')
                # print(aa,end='|')
                # print(b,end='|')
                # print(bb)
        #print(i,'번째 리스트:',keywords_search)
        #print('-'*10)
        
        if len(big_search) != 0:
            df['total_big'][i] = 1
            
        else:
            df['total_big'][i] = 0
        
        if len(small_search) != 0:
            df['total_small'][i] = 1
            
        else:
            df['total_small'][i] = 0
    
    #df.rename(columns = {'keyword_column_name':keyword_column_name},inplace=True)
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return df

In [164]:
df_keywords['total_keyword'].head(5)
print(len(df_keywords['total_keyword']))

2000


In [160]:
df_test_bs = total_big_small(big,small,df_keywords)

100%|██████████| 2000/2000 [00:12<00:00, 161.36it/s]

12.40469 sec


In [168]:
df_test_bs[df_test_bs['total_small'] == 1]

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이,review,total_keyword,chongjang_keyword,chest_keyword,shoulder_keyword,arm_keyword,total_big,total_small
31,LV 4 핫맨맨맨,남성,178.0,75.0,SIGN LOGO HOOD BLACK,XL,5.0,평소 상의 105 입는데 살짝 루즈핏에 넉넉하고 좋네요. 재질도 탄탄하고 가성비 좋아요~~,보통이에요,보통이에요,선명해요,보통이에요,XL,74.0,65.0,63.0,57.0,평소 상의 입는데 살짝 루즈핏에 넉넉하고 좋네요 재질도 탄탄하고 가성비 좋아요,"핏에 넉넉하고 좋네요,루즈핏,상의 입는데 살짝 루즈핏에,평소 상의",0,0,0,0,1,1
36,LV 4 사자사자저거사자,남성,172.0,71.0,SIGN LOGO HOOD BLACK,L,5.0,아들 한테 이옷 뺏겨서 한치수 작은거로 저도 다시 삿어요. 예쁘고 편한지 자주 입고 다니네요.쎄일할때 사서 가격도 저렴하게 잘사서 잘입고 다니네요.학원갈때나 운동하러 나갈때 입고가요,커요,보통이에요,선명해요,보통이에요,L,72.0,64.0,61.0,56.0,아들 한테 이옷 뺏겨서 한치수 작은거로 저도 다시 삿어요 예쁘고 편한지 자주 입고 다니네요쎄일할때 사서 가격도 저렴하게 잘사서 잘입고 다니네요학원갈때나 운동하러 나갈때 입고가요,"옷 뺏겨서 한치수 작은거로,이옷",0,0,0,0,0,1
53,LV 4 뉴비_8d40f92a41ff,남성,179.0,50.0,SIGN LOGO HOOD GRAY,M,5.0,기본후드티에 사이즈 원래보다 작게 시킨게잘맞네요,보통이에요,보통이에요,보통이에요,보통이에요,M,70.0,63.0,59.0,55.0,기본후드티에 사이즈 원래보다 작게 시킨게잘맞네요,"사이즈 원래보다 작게 시킨게잘맞네요,기본후드티에 사이즈,후드티에 사이즈 원래보다,기본후드,후드티에 사이즈 원래보다,기본후드티",0,0,0,0,0,1
119,LV 3 애솜솜이,남성,180.0,64.0,(23SS) 2 TONE ARCH HOODIE GRAY,L,4.0,키 180에 몸무게 64 나가는 남자친구한테 입혀보았습니다품은 작아보이고 길이도 짧아보입니다생각보다 더 크게 사세염,작아요,밝아요,선명해요,두꺼워요,L,69.0,60.0,61.0,63.0,키 에 몸무게 나가는 남자친구한테 입혀보았습니다품은 작아보이고 길이도 짧아보입니다생각보다 더 크게 사세염,"품은 작아보이고 길이도,입혀보았습니다품","길이도 짧아보입니다생각보다 더,작아보이고 길이",0,0,0,1,1
133,LV 2 소똥이대마왕,남성,178.0,77.0,(23SS) 2 TONE ARCH HOODIE BLACK,XL,5.0,평소에 xl 입는데 더 오버핏으로 입으려면한사이즈업 해도 될듯함요그래도 무난하게 잘 입고다닐듯 옷 이뻐요^^,보통이에요,보통이에요,보통이에요,보통이에요,XL,71.0,62.0,63.0,64.0,평소에 xl 입는데 더 오버핏으로 입으려면한사이즈업 해도 될듯함요그래도 무난하게 잘 입고다닐듯 옷 이뻐요,"사이즈업 해도 될듯함요그래도,입으려면한사이즈,핏으로 입으려면한사이즈업 해도,오버핏,입고다닐듯 옷,핏으로 입으려면한사이즈업 해도 될듯함요그래도,오버핏으로",0,0,0,0,1,1
149,LV 3 _lisianthus.x,여성,172.0,52.0,(23SS) 2 TONE ARCH HOODIE GRAY,L,5.0,클론템 한번사봣읍니다 사이즈가 작게나왓는지 원래 m입는데 l이 딱좋네용 굿,보통이에요,보통이에요,보통이에요,보통이에요,L,69.0,60.0,61.0,63.0,클론템 한번사봣읍니다 사이즈가 작게나왓는지 원래 m입는데 l이 딱좋네용 굿,"사이즈가 작게나왓는지 원래,한번사봣읍니다 사이즈",0,0,0,0,0,1
199,LV 4 뵤리1990,남성,170.0,75.0,[Mmlg] 19MG HOOD (AUTHENTIC NAVY),M,4.0,팔이길고 전체적인 길이감은 짧아서 탐고해야할듯해요,보통이에요,보통이에요,보통이에요,보통이에요,M,67.0,56.0,63.0,64.0,팔이길고 전체적인 길이감은 짧아서 탐고해야할듯해요,"전체적인 길이감은 짧아서,팔이길고 전체","길이감은 짧아서 탐고해야할듯해요,전체적인 길이,길이감은 짧아서 탐고해야할듯해요,전체적인 길이감",0,0,"팔이길고 전체적인 길이감은,팔이길고 전체적인 길이감은",1,1
200,LV 4 뵤리1990,남성,170.0,75.0,[Mmlg] 19MG HOOD (EVERY GREY),M,4.0,팔이길고 전체적인 길이감은 짧아서 탐고해야할듯합니다,보통이에요,보통이에요,보통이에요,보통이에요,M,67.0,56.0,63.0,64.0,팔이길고 전체적인 길이감은 짧아서 탐고해야할듯합니다,"전체적인 길이감은 짧아서,팔이길고 전체","길이감은 짧아서 탐고해야할듯합니다,전체적인 길이,길이감은 짧아서 탐고해야할듯합니다,전체적인 길이감",0,0,"팔이길고 전체적인 길이감은,팔이길고 전체적인 길이감은",1,1
205,LV 3 d.gyu__xx,남성,171.0,55.0,[Mmlg] 19MG HOOD (AUTHENTIC NAVY),M,5.0,전에 샀는데 마음에 들어서 또 구매합니다 한사이즈 작게 사는거 추천드려요!!,보통이에요,밝아요,선명해요,보통이에요,M,67.0,56.0,63.0,64.0,전에 샀는데 마음에 들어서 또 구매합니다 한사이즈 작게 사는거 추천드려요,"사이즈 작게 사는거 추천드려요,한사이즈",0,0,0,0,0,1
243,LV 4 elly88,남성,178.0,75.0,[Mmlg] 19MG HOOD (REDBEAN),L,4.0,기모인데도 많이 두껍지 않아요팔길이는 좋은데 그에 비해 몸통 길이가 짧아서 시보리가 움직이면 튀어나와 핏이 아쉬웠어요.. 그래도 편한 맛에 입으려구요!,보통이에요,보통이에요,보통이에요,보통이에요,L,70.0,59.0,66.0,67.0,기모인데도 많이 두껍지 않아요팔길이는 좋은데 그에 비해 몸통 길이가 짧아서 시보리가 움직이면 튀어나와 핏이 아쉬웠어요 그래도 편한 맛에 입으려구요,"몸통 길이가 짧아서 시보리가,비해 몸통,핏이 아쉬웠어요 그래도,튀어나와 핏,핏이 아쉬웠어요 그래도 편한,튀어나와 핏이","길이는 좋은데 그에,길이가 짧아서 시보리가,않아요팔길이,몸통 길이","통 길이가 짧아서 시보리가,몸통,몸통 길이가 짧아서 시보리가,비해 몸통","통 길이가 짧아서 시보리가,몸통,몸통 길이가 짧아서 시보리가,비해 몸통","팔길이는 좋은데 그에,않아요팔길이,팔길이는 좋은데 그에,않아요팔길,팔길이는 좋은데 그에,않아요팔,요팔길이는 좋은데 그에,않아요팔",1,1


In [ ]:
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [160]:
len(test)

34309

In [ ]:
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review.encode('utf-8')):
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))

    return feature_temp


def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list


def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        # for negative in negative_word_emotion:
        #     if re.findall(negative,ngram):
        #         is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        # for negative in negative_word_emotion:
        #     if re.findall(negative,ngram):
        #         is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

## 비고

In [128]:
#쓸지말지 고민
# #최종 사이즈 사전 완성
def final_sizedict(size_figure_list):
    all_big = {}
    all_small = {}
    for i in size_figure_list:
        all_big[i] = big
    
    for i in size_figure_list:
        all_small[i] = small
    
    return all_big,all_small

In [129]:
total_big,total_small = final_sizedict(total)
chongjang_big,chongjang_small = final_sizedict(chongjang)
shoulder_big,shoulder_small = final_sizedict(shoulder)
arm_big,arm_small = final_sizedict(arm)
chest_big,chest_small = final_sizedict(chest)


In [133]:
chest_small

{'가슴팍': ['좁은',
  '사이즈업',
  '길었으면',
  '크롭하긴',
  '좁다고',
  '짧은',
  '숏',
  '짧고',
  '작긴',
  '짧으면',
  '크롭하',
  '크롭해',
  '크롭느낌',
  '작기는',
  '크롭이라',
  '크롭이',
  '짧아서',
  '숏한',
  '작으면',
  '짧습니다',
  '달라붙는',
  '작음',
  '작은',
  '짧아',
  '작지',
  '짧지',
  '짧긴',
  '크롭된',
  '작아',
  '작게',
  '작네',
  '좁다는',
  '짧',
  '짝',
  '작은것',
  '작아요',
  '짧아요',
  '미니',
  '짧은데',
  '작은데',
  '작았으면',
  '좁게',
  '짧지만',
  '좁',
  '짧았어요',
  '작아서',
  '좁아요',
  '좁고',
  '작아도',
  '작아진',
  '좁아',
  '짧았지만',
  '작다고',
  '쪼여서',
  '작',
  '짧네요',
  '타이트',
  '숏하',
  '사이즈업하',
  '좁아서',
  '짧다는',
  '작고',
  '컸으면',
  '짧음',
  '쪼이는',
  '붙어서',
  '짧게',
  '크롭한',
  '작습니다',
  '작았어요',
  '크롭',
  '쫄려',
  '작다',
  '짧다고',
  '핏되',
  '작네요'],
 '몸통': ['좁은',
  '사이즈업',
  '길었으면',
  '크롭하긴',
  '좁다고',
  '짧은',
  '숏',
  '짧고',
  '작긴',
  '짧으면',
  '크롭하',
  '크롭해',
  '크롭느낌',
  '작기는',
  '크롭이라',
  '크롭이',
  '짧아서',
  '숏한',
  '작으면',
  '짧습니다',
  '달라붙는',
  '작음',
  '작은',
  '짧아',
  '작지',
  '짧지',
  '짧긴',
  '크롭된',
  '작아',
  '작게',
  '작네',
  '좁다는',
  '짧',
  '짝',
  '작은것',
  '작아요',
  '짧아요',
  '미

In [ ]:
def get_front_keywords_back(feature_keywords, review):
    
    feature_temp = []
    
    
    for i in range(len(review)):
        for keyword in feature_keywords:
            if re.findall(keyword, review[i]):
                a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                
                for ngram in a:
                    t = ()
                    feature_temp.append(t + (ngram,keyword))
                for ngram in b:
                    t = ()
                    feature_temp.append(t + (ngram,keyword))
                for ngram in c:
                    t = ()
                    feature_temp.append(t + (ngram,keyword))

    return feature_temp

In [ ]:
# # #최종 사이즈 사전 완성
# # def final_sizedict(size_figure_list):
# #     all_big = {}
# #     all_small = {}
# #     for i in size_figure_list:
# #         all_big[i] = big
    
# #     for i in size_figure_list:
# #         all_small[i] = small
    
# #     return all_big,all_small
# # %%
# df = crawlingdataconcat()
# df = crawlingdataprocessing(df)
# df = add_reviewcol(df)
# #df.to_csv('data/hood_1to20.csv', encoding="UTF-8", index=False) #파일로 저장
# # %%
# total_big,total_small           = final_sizedict(total)
# chongjang_big,chongjang_small   = final_sizedict(chongjang)
# shoulder_big,shoulder_small     = final_sizedict(shoulder)
# arm_big,arm_small               = final_sizedict(arm)
# chest_big,chest_small           = final_sizedict(chest)